In [2]:
### ficher notebook pour la visualisation###
!pip install selenium
!pip install webdriver-manager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--verbose') 

In [37]:
from selenium.webdriver.chrome.service import Service
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

browser = webdriver.Chrome(service=service,options=chrome_options)

Scrapping des matches 

In [53]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

def get_page_content(url, wait_time=10):
    # Utiliser Selenium pour laisser le temps à la page de charger
    driver = webdriver.Chrome()
    driver.get(url)

    # Attendre que la page se charge
    time.sleep(wait_time)

    # Utiliser requests pour obtenir le contenu de la page après le chargement
    page_content = driver.page_source

    # Fermer le navigateur Selenium
    driver.quit()

    return page_content




In [ ]:
# Exemple d'utilisation avec l'URL du tournoi ATP que vous avez fourni
tournament_url = 'https://www.atptour.com/en/scores/archive/adelaide/2843/2023/results'
matches_df = scrape_tournament_matches(tournament_url)

# Afficher le DataFrame avec les informations des matches
print(matches_df)

In [63]:

def scrape_tournament_matches(url):
    # Obtenir le contenu de la page avec Selenium et requests
    page_content = get_page_content(url)

    # Utiliser BeautifulSoup pour extraire les informations
    soup = BeautifulSoup(page_content, 'html.parser')

    matches = []
    
    # Extraire le nom du tournoi et les dates
    tournament_info = soup.find('td', class_='title-content')
    tournament_name = tournament_info.find('a', class_='tourney-title').text.strip()
    tournament_dates = tournament_info.find('span', class_='tourney-dates').text.strip()

    tourney_details_table = soup.find('td', class_='tourney-details-table-wrapper')

    prize_money_detail = tourney_details_table.find('td', class_='tourney-details prize-money')
    prize_money_value = prize_money_detail.find('span', class_='item-value').text.strip() if prize_money_detail else None


    # Trouver tous les blocs de matches
    match_blocks = soup.find_all('div', class_='day-table-wrapper')

    for match_block in match_blocks:
        # Récupérer le nom du tour à partir du th précédent
        tour_name_element = match_block.find_previous('th')
        tour_name = tour_name_element.text.strip() if tour_name_element else 'Unknown Tour'

        # Trouver tous les matches dans ce bloc
        match_rows = match_block.find_all('tr')

        for match_row in match_rows:
            # Récupérer les informations du match
            players_element = match_row.find_all('td', class_='day-table-name')
            result_element = match_row.find('td', class_='day-table-vertical-label')
            winner_element = match_row.find('td', class_='day-table-button')
            score_element = match_row.find('td', class_='day-table-score')

            # Vérifier si les éléments existent avant d'extraire le texte
            if players_element and result_element and winner_element:
                players = [a.text.strip() for a in players_element]
                result = result_element.text.strip()
                winner = players[0]  # Le gagnant est le premier joueur
                loser = players[1]   # Le perdant est le deuxième joueur

                # Ajouter les informations au tableau
                match_info = {'Tournament': tournament_name, 'Dates': tournament_dates,  'Prize':prize_money_value, 'Tour': tour_name, 'Winner': winner, 'Loser': loser}

                # Nouveau code pour extraire le score
                score_link = match_row.find('a', href=re.compile("/en/scores/stats-centre/archive/"))
                if score_link:
                    # Extraire le texte à l'intérieur de la balise a (contient le score)
                    score_text = score_link.get_text(strip=True)
                    match_info['Score'] = score_text

                matches.append(match_info)
                print(f"Match: {match_info}")
            else:
                print("Certaines informations manquent pour ce match.")

    # Créer un DataFrame Pandas avec les informations extraites
    matches_df = pd.DataFrame(matches)

    return matches_df

# Exemple d'utilisation avec l'URL du tournoi ATP que vous avez fourni
tournament_url = 'https://www.atptour.com/en/scores/archive/adelaide/2843/2023/results'
matches_df = scrape_tournament_matches(tournament_url)

# Afficher le DataFrame avec les informations des matches
print(matches_df)

Certaines informations manquent pour ce match.
Match: {'Tournament': 'Adelaide International 1', 'Dates': '2023.01.01 - 2023.01.08', 'Prize': '$642,735', 'Tour': 'Unknown Tour', 'Winner': 'Novak Djokovic', 'Loser': 'Sebastian Korda', 'Score': '67876364'}
Certaines informations manquent pour ce match.
Match: {'Tournament': 'Adelaide International 1', 'Dates': '2023.01.01 - 2023.01.08', 'Prize': '$642,735', 'Tour': 'Unknown Tour', 'Winner': 'Novak Djokovic', 'Loser': 'Daniil Medvedev', 'Score': '6364'}
Match: {'Tournament': 'Adelaide International 1', 'Dates': '2023.01.01 - 2023.01.08', 'Prize': '$642,735', 'Tour': 'Unknown Tour', 'Winner': 'Sebastian Korda', 'Loser': 'Yoshihito Nishioka', 'Score': '76510 \n                                            \n                                            \n                                            \n\n\n(RET)'}
Certaines informations manquent pour ce match.
Match: {'Tournament': 'Adelaide International 1', 'Dates': '2023.01.01 - 2023.01.08', 'P

In [64]:
import re
import pandas as pd
from bs4 import BeautifulSoup

def scrape_tournament_rounds(url):
    # Obtenir le contenu de la page avec Selenium et requests
    page_content = get_page_content(url)

    # Utiliser BeautifulSoup pour extraire les informations
    soup = BeautifulSoup(page_content, 'html.parser')

    rounds_info = []
    
    # Trouver tous les blocs de rounds
    round_blocks = soup.find_all('div', class_='day-table-wrapper')

    for round_block in round_blocks:
        # Trouver tous les éléments th à l'intérieur de la balise thead
        round_elements = round_block.select('thead th[colspan]')

        # Récupérer le texte de chaque élément th
        round_names = [element.text.strip() for element in round_elements]

        # Ajouter les informations au tableau
        round_info = {'Rounds': round_names}

        rounds_info.append(round_info)

    # Créer un DataFrame Pandas avec les informations extraites
    rounds_df = pd.DataFrame(rounds_info)

    return rounds_df



